In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

In [1]:
# We need this to run the tutorial directly in the jupyter notebook
ref_paths = {}
base_path = "vasp/rss_Si_small/"

for i in range(20):
    ref_paths[f"static_bulk_{i}"] = f"{base_path}static_bulk_{i}"

ref_paths["static_isolated_0"] = f"{base_path}static_isolated_0"

We are using a config file. You can take a look at it here:

In [2]:
from autoplex.settings import RssConfig
rss_config = RssConfig.from_file('rss_si_config.yaml')
print(rss_config)

tag='Si' train_from_scratch=True resume_from_previous_state=ResumeFromPreviousState(test_error=None, pre_database_dir=None, mlip_path=None, isolated_atom_energies=None) generated_struct_numbers=[1000] buildcell_options=[BuildcellOptions(ABFIX=False, NFORM='1', SYMMOPS='1-4', SYSTEM=None, SLACK=0.25, OCTET=False, OVERLAP=0.1, MINSEP=None, NATOM='{6,8,10,12,14,16,18,20,22,24}')] fragment_file=None fragment_numbers=None num_processes_buildcell=128 num_of_initial_selected_structs=[20] num_of_rss_selected_structs=10 initial_selection_enabled=True rss_selection_method='bcur2i' bcur_params=BcurParams(soap_paras=SoapParas(l_max=12, n_max=12, atom_sigma=0.0875, cutoff=10.5, cutoff_transition_width=1.0, zeta=4.0, average=True, species=True), frac_of_bcur=0.8, bolt_max_num=3000) random_seed=None include_isolated_atom=True isolatedatom_box=[20.0, 20.0, 20.0] e0_spin=False include_dimer=False dimer_box=[20.0, 20.0, 20.0] dimer_range=[1.0, 5.0] dimer_num=21 custom_incar=CustomIncar(ISMEAR=0, SIGMA=0

In [3]:
from autoplex.auto.rss.flows import RssMaker
from atomate2.vasp.powerups import update_user_incar_settings
from atomate2.vasp.powerups import update_vasp_custodian_handlers
from jobflow import Flow


rss_job = RssMaker(name="rss", rss_config=rss_config).make()

autoplex_flow = update_user_incar_settings(Flow(jobs=[rss_job], output=rss_job.output), {"NPAR": 4})

autoplex_flow = update_vasp_custodian_handlers(autoplex_flow, custom_handlers={})

autoplex_flow.name = "new test rss"

/home/jgeorge/miniconda3/envs/2025_01_06_Fix_automated_finetuning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jgeorge/miniconda3/envs/2025_01_06_Fix_automated_finetuning/lib/python3.11/site-packages/nequip/__init__.py:20: UserWarning: !! PyTorch version 2.2.1+cu121 found. Upstream issues in PyTorch versions 1.13.* and 2.* have been seen to cause unusual performance degredations on some CUDA systems that become worse over time; see https://github.com/mir-group/nequip/discussions/311. The best tested PyTorch version to use with CUDA devices is 1.11; while using other versions if you observe this problem, an unexpected lack of this problem, or other strange behavior, please post in the linked GitHub issue.
  warnings.warn(


In [4]:
from mock_vasp import mock_vasp
from jobflow import run_locally

with mock_vasp(ref_paths=ref_paths, clean_folders=True) as mf:
    run_locally(
        autoplex_flow,
        create_folders=True,
        ensure_success=True,
        raise_immediately=True,
    )

2025-02-20 16:32:12,293 INFO Started executing jobs locally


2025-02-20 16:32:12,293 - INFO - Started executing jobs locally


2025-02-20 16:32:12,299 INFO Starting job - rss (b36a55bb-2f22-47ea-8bfb-c919e92846ba)


2025-02-20 16:32:12,299 - INFO - Starting job - rss (b36a55bb-2f22-47ea-8bfb-c919e92846ba)


2025-02-20 16:32:12,514 INFO Finished job - rss (b36a55bb-2f22-47ea-8bfb-c919e92846ba)


...

[{'ABFIX': False, 'NFORM': '1', 'SYMMOPS': '1-4', 'SLACK': 0.25, 'OCTET': False, 'OVERLAP': 0.1, 'NATOM': '{6,8,10,12,14,16,18,20,22,24}'}]
2025-02-20 16:32:12,520 INFO Finished job - initial_rss (b7a0a2ab-6044-45d9-bd9d-76527a1e991c)


2025-02-20 16:32:12,520 - INFO - Finished job - initial_rss (b7a0a2ab-6044-45d9-bd9d-76527a1e991c)


2025-02-20 16:32:12,524 INFO Starting job - do_randomized_structure_generation (997f3881-0efb-4910-bd06-1193014fc61d)


2025-02-20 16:32:12,524 - INFO - Starting job - do_randomized_structure_generation (997f3881-0efb-4910-bd06-1193014fc61d)


RuntimeError: RSS flows requires the executable 'buildcell' to be in PATH. Please follow the instructions in the autoplex documentation to install the AIRSS library and add it to PATH. Link to the documentation: https://autoatml.github.io/autoplex/user/index.html#enabling-rss-workflows